# 4.3 Two Examples

## Exercises 4.3

In [1]:
%%capture
%run ./4_2_rsa_algorithm.ipynb
from abc import ABC, abstractmethod
from typing import TypeVar, Tuple, Annotated, Generic
from typing_extensions import Literal
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPrivateKey
from src.helpers import format_pk_plaintext
from __main__ import crack_trivial_rsa

N = TypeVar('N')  # For specifying character block size
CharStr = Annotated[str, Literal[1]]  # For single-character strings

> The purpose of the first two problems is to give you an opportunity to practice enciphering and deciphering using the RSA algorithm without the use of a computer. You should be able to do these problems with no more than a calculator.

Um. yeah.. no.   Working through the problem here in a notebook is as close as I'm getting to doing it by hand with calculator.  I don't need to experience that.

I'm going to define an abstract class to represent the correspondence and encoding scheme and then I can implement the encryption and decryption independent of it.

In [2]:
class EncodingScheme(Generic[N], ABC):
    """
    Abstract base class for an encoding scheme.
    Concreate subclasses define the length of the character blocksize
    and the correspondence between characters and numbers.
    """
    @abstractmethod
    def encode(self, chars: Tuple[CharStr, N]) -> int:
        """ Turn a block of characters into a single number. """
        pass
    @abstractmethod
    def decode(self, encoded_chars: int) -> Tuple[CharStr, N]:
        """ Turn a number into a block of characters. """
        pass
    @abstractmethod
    def block_size(self) -> int:
        """ Return the character blocksize used."""
        pass

With the encoder interface defined, we can define the `pk_encrypt` and `pk_decrypt` methods

In [3]:
def pk_encrypt(e: int, n: int, encoder: EncodingScheme, message: str) -> list[int]:
    """ Encrypt a message using the public key (e, n) with the given encoder. """
    blocksize = encoder.block_size()
    # pad message so that is an even multiple of blocksize
    message += ' ' * (-len(message) % blocksize)
    result = []
    for i in range(0, len(message), blocksize):
        block = tuple(str(char) for char in message[i:i+blocksize])
        result.append(pow(encoder.encode(block), e, n))
    return result

In [4]:
def pk_decrypt(d: int, n: int, encoder: EncodingScheme, ciphertext: list[int]) -> str:
    """ Decrypt a message using the private key (d, n) with the given encoder. """
    msg_chars = []
    for encrypted_block in ciphertext:
        block = encoder.decode(pow(encrypted_block, d, n))
        msg_chars.extend(block)
    return ''.join(msg_chars)

Note that questions 1, and 2, below use the same encoding scheme, so let's define that.

In [5]:
Tuple2Chars = Tuple[CharStr, Literal[2]]
class SimpleEncoder(EncodingScheme[Literal[2]]):
    _CHARS = "\0abcdefghijklmnopqrstuvwxyz' ?" # using \0 makes _CHARS.index('a') = 1
    _default_char = ' '
    def _pos(self, char: CharStr) -> int:
        return self._CHARS.index(char) if char in self._CHARS else self._CHARS.index(self._default_char)
    def _char(self, pos: int) -> CharStr:
        return self._CHARS[pos] if  0 < pos < len(self._CHARS) else self._default_char
    def encode(self, chars: Tuple2Chars) -> int:
        return self._pos(chars[0].lower()) * 100 + self._pos(chars[1].lower())
    def decode(self, encoded_chars: int) -> Tuple2Chars:
        return self._char(encoded_chars // 100), self._char(encoded_chars % 100)
    def block_size(self) -> int:
        return 2
encoder_1_2 = SimpleEncoder()

### 1. The message: `Why isn't phonetic spelled the way it sounds?`

* **The keys:** $n = 3363$, $e = 5$.
* **The correspondence:** Associate each letter with its position in the alphabet. Associate the apostrophe with the number $27$, the space with the number $28$, and the question mark with the number $29$.
* **The task:** Combine pairs of numbers as in the examples and encipher the message using the RSA algorithm.

In [6]:
msg = "Why isn't phonetic spelled the way it sounds?"
ciphertext = pk_encrypt(5, 3363, encoder_1_2, msg)
print(ciphertext)

[700, 305, 3241, 3167, 1929, 2877, 2579, 904, 801, 2177, 2448, 306, 3138, 2862, 1531, 2724, 995, 2740, 1929, 3175, 1244, 932, 2673]


In [7]:
print(pk_decrypt(crack_trivial_rsa(3363, 5), 3363, encoder_1_2, ciphertext))

why isn't phonetic spelled the way it sounds? 


### 2. The enciphered message: `[239, 1093, 3145, 2258, 336, 3746, 1098, 3712, 637, 1528, 259, 1633, 2258, 1902, 427, 1048, 1637, 1198, 1694]`

* **The keys:** $n = 3953$, $d = 5$.
* **The task:** Decipher this message using the RSA algorithm. Then decompose each of the integers in the deciphered message into two integers as in the examples. (Be careful when it comes to 3-digit numbers!!)
* **The correspondence:** Associate each number between $1$ and $26$ with the letter occupying that alphabetic position. Associate the number $27$ with the apostrophe, the number $28$ with the space, and the number $29$ with the question mark.

In [8]:
ciphertext = [239, 1093, 3145, 2258, 336, 3746, 1098, 3712, 637, 1528, 259, 1633, 2258, 1902, 427, 1048, 1637, 1198, 1694]
plaintext = pk_decrypt(5, 3953, encoder_1_2, ciphertext)
print(plaintext)

why is abbreviation such a long word? 


In [9]:
print(pk_encrypt(crack_trivial_rsa(3953, 5), 3953, encoder_1_2, plaintext))

[239, 1093, 3145, 2258, 336, 3746, 1098, 3712, 637, 1528, 259, 1633, 2258, 1902, 427, 1048, 1637, 1198, 1694]


### 3.Encipher the following message using the RSA algorithm with $n = 34618195959169$ and $e = 20000000089$:

```
 Why is it that when you transport something by car, it's
 called a shipment, but when you transport something by ship,
 it's called cargo?
```

 Use a text-numeric correspondence of your choice. Check your answer by deciphering it using $d = 4771730348713$.

In [10]:
# I checked and n=34618195959169 requires 46 bits, so we can use block size of
# 5 characters and represent each as an 8 bit ascii code.
AsciiEncoderTupleChars = Tuple[CharStr, Literal[5]]
class AsciiEncoder(EncodingScheme[Literal[5]]):
    def __init__(self):
        self._BLOCKSIZE = 5  # make blocksize easier to change for larger n
    def encode(self, chars: AsciiEncoderTupleChars) -> int:
        if len(chars) != self._BLOCKSIZE:
            raise ValueError(f"chars must be of length {self._BLOCKSIZE}, not {len(chars)}")
        # Take each of the chars, get its ascii code and build the block
        # block is reversed so we can shift out more naturally during decode
        result = 0
        for i in range(len(chars) - 1, -1, -1):  # reverse order
            if ord(chars[i]) > 255:
                # raise an error if input contains wide characters
                raise ValueError(f"character {chars[i]} is not in range(256)")
            result = (result << 8) | (ord(chars[i]) & 0xFF)
        return result
    def decode(self, encoded_chars: int) -> AsciiEncoderTupleChars:
        chars = []
        for _ in range(self._BLOCKSIZE):
            chars.append(chr(encoded_chars & 0xFF))
            encoded_chars >>= 8
        return tuple(chars)
    def block_size(self) -> int:
        return self._BLOCKSIZE
encoder_3 = AsciiEncoder()

In [11]:
msg_3 = "Why is it that when you transport something by car, it's " + \
        "called a shipment, but when you transport something by ship, " + \
        "it's called cargo?"
n = 34618195959169
e = 20000000089
ciphertext_3 = pk_encrypt(e, n, encoder_3, msg_3)
print(ciphertext_3)

[21026755237750, 3673500579484, 26134721829671, 23033184621597, 14904862462988, 26826051023443, 17187528846116, 14225599331655, 11720127759526, 17009559551858, 3599980615836, 29499074929590, 21980861814084, 4899869418213, 362102174095, 9825773672909, 23033184621597, 14904862462988, 26826051023443, 17187528846116, 14225599331655, 11720127759526, 14827570556748, 991926326256, 6134789661838, 22947818031193, 2192425791530, 7418393140983]


In [12]:
d = 4771730348713
plaintext_3_prime = pk_decrypt(d, n, encoder_3, ciphertext_3)
print(format_pk_plaintext(plaintext_3_prime))

Why is it that when you transport something by car, it's cal
led a shipment, but when you transport something by ship, it
's called cargo?    


### 4. Decipher the following message using the RSA algorithm with $n = 41378299599863$ and $d = 17927688850145$.

```
 [31041840486988, 29077957566727, 11122505017205,
 40696280438081,33635577577494, 23781081595165, 13392200113552,
 13042556080033, 12862388751593, 34364195535462,
 2445939988968, 33291530313321,41132626495683, 34138477928943,
 5921732609276, 12675552190449, 3052697916053, 20979766863114,
 33068565954829, 29024930510830, 29540937197477.
 25212533372768, 5204853349407, 1812489541036, 11924887995293,
 22111140271445, 6327640502644, 22111140271445, 26465841739700,
 39782824809293, 24869716524496, 35280750488903,
 39782824809293, 29077957566727, 11122505017205, 5921732609276,
 14670735876808, 6793151864986, 32428301166222, 5921732609276,
 12675552190449, 34601196283821, 26850267312878, 35401622221039,
 9312542831382, 35258512628692, 2274436743060].
```

Each integer in this message represents two integers in deciphered form. Convert your deciphered message into text using the correspondence between the integers and the position of the characters in the text string:

```
 abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ 1234567890-=!@#$%^&*()_+[]\{}|';,:./<?>~
```

(Note that the space character appears between the 'Z' and the '1', i.e., in position 53.)

In [13]:
class CommonEncoder(EncodingScheme[Literal[2]]):
    _CHARS = "\0abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ 1234567890-=!@#$%^&*()_+[]\\{}|';,:./<?>~"
    _default_char = ' '
    def _pos(self, char: CharStr) -> int:
        return self._CHARS.index(char) if char in self._CHARS else self._CHARS.index(self._default_char)
    def _char(self, pos: int) -> CharStr:
        return self._CHARS[pos] if  0 < pos < len(self._CHARS) else self._default_char
    def encode(self, chars: Tuple2Chars) -> int:
        return self._pos(chars[0].lower()) * 100 + self._pos(chars[1].lower())
    def decode(self, encoded_chars: int) -> Tuple2Chars:
        return self._char(encoded_chars // 100), self._char(encoded_chars % 100)
    def block_size(self) -> int:
        return 2
encoder_4 = CommonEncoder()

In [14]:
msg_4 = [31041840486988, 29077957566727, 11122505017205, 40696280438081,
         33635577577494, 23781081595165, 13392200113552, 13042556080033,
         12862388751593, 34364195535462,  2445939988968, 33291530313321,
         41132626495683, 34138477928943,  5921732609276, 12675552190449,
          3052697916053, 20979766863114, 33068565954829, 29024930510830,
         29540937197477, 25212533372768,  5204853349407,  1812489541036,
         11924887995293, 22111140271445,  6327640502644, 22111140271445,
         26465841739700, 39782824809293, 24869716524496, 35280750488903,
         39782824809293, 29077957566727, 11122505017205,  5921732609276,
         14670735876808,  6793151864986, 32428301166222,  5921732609276,
         12675552190449, 34601196283821, 26850267312878, 35401622221039,
          9312542831382, 35258512628692, 2274436743060]
n = 41378299599863
d = 17927688850145
plaintext_4 = pk_decrypt(d, n, encoder_4, msg_4)
print(format_pk_plaintext(plaintext_4))

If you're in a vehicle going the speed of light, what happen
s when you turn on the headlights?



### 5. As a trusted person, you have been contracted by a small clandestine organization to generate a set of six public and private keys. Money is no object. They have agreed to pay you an amount proportional to the size of n. Go for it.

There is no sense making keys that people can't readily use, so let's go with 8192 bit keys.  Since the time the textbook was written, the generation of RSA keys has become a commonplace task and good library supposrt is available to create and use keys of this magnitude.  We'll make use of that support for this exercise.

In [15]:
def generate_key() -> RSAPrivateKey:
    """Generates an RSA private key."""
    return rsa.generate_private_key(
        # this is a Fermat prime and is efficient to use as a encrypting exponent
        # n and d will be randomized based on this value
        public_exponent=65537,
        key_size=8192
    )
key = None
for _ in range(6):
    key = generate_key()
    public_numbers = key.public_key().public_numbers()
    private_numbers = key.private_numbers()
    print("-" * 60)
    print(f"\tn = {public_numbers.n}")
    print(f"\te = {public_numbers.e}")
    print(f"\td = {private_numbers.d}")


------------------------------------------------------------
	n = 23787370777055235646432663187497533325594000825070331865899531044795688090613071838914587497358761284155170622793891754720451682743262397249899810420459703725186795423511160562977839872776453481554777373666966006871313114763113179210915240892529947120474201524846643904371925111897213629375875442719855410768613877081008110069403027282749365710810221430832097580370781753399396283768705882588042279830196951837063651443115748678492431384167848324777253383076611295103318577590985335816353687181197188657858983901950050269982901119180158733843956372245209660791019019981327127013100962628691798157289609902738079627939
	e = 65537
	d = 118935039080602707121519982197524935839764441313441969358041386281927019447519590157857882282291207983154039850421936466222097554073458082164993360675286261450313702555444055926828792430404020276116841628441817921656344987496786953382558037988681402451088489794493923655104634338869325145702227216235

In [16]:
number_of_n_digits = len(str(key.public_key().public_numbers().n))
print(number_of_n_digits)

617


Just for reference, generating all six of these keys took less than a minute, and the values of $n$ and $d$ have over $2465$ decimal digits.  So picking a remuneration rate of \$1 per decimal digit of $n$ per key, that will be:

In [17]:
invoice_amount = ( 1  *    # dollar per decimal digit of n
                   number_of_n_digits *  # digits of n per key
                   6)      # keys
print(f"That will be ${invoice_amount:,} please.  Thank you. :-)")

That will be $3,702 please.  Thank you. :-)


Not bad for 1 minute of work.